In [35]:
import json
import os
import pickle

import numpy as np
from skimage import feature, io, transform, util, filters, morphology, measure, color
from sklearn import model_selection

In [36]:
DATA_DIRECTORY = "data"
MEN_DIRECTORY = os.path.join(DATA_DIRECTORY, "men")
WOMEN_DIRECTORY = os.path.join(DATA_DIRECTORY, "women")
LABELS_FILENAME = os.path.join(DATA_DIRECTORY, "labels.jsonl")

In [37]:
paths: list[str] = []
labels: list[int] = []
with open(LABELS_FILENAME, "r") as f:
    for line in f:
        entry = json.loads(line)
        path = entry["image_url"]
        label = int(entry["label"])
        paths.append(path)
        labels.append(label)

In [38]:
path = paths[0]
label = labels[0]

In [39]:
image = io.imread(path)
grayscale = color.rgb2gray(image)

In [40]:
threshold = filters.threshold_otsu(grayscale)
binary = grayscale > threshold

In [41]:
cleaned = morphology.remove_small_objects(binary, min_size=64, connectivity=2)
filled = morphology.remove_small_holes(cleaned, area_threshold=64, connectivity=2)

In [42]:
labels = morphology.label(filled)
regions = measure.regionprops(labels)
region = max(regions, key=lambda region: region.area)
label = region.label

In [43]:
hand = np.zeros_like(grayscale)
hand[labels == label] = grayscale[labels == label]

In [44]:
binary_hand = ~(hand > 0)
hand_region = measure.regionprops(binary_hand.astype(int))[0]
hull = morphology.convex_hull_image(binary_hand)

In [45]:
cropped = image[hand_region.bbox[0] : hand_region.bbox[2], hand_region.bbox[1] : hand_region.bbox[3]]